<a href="https://colab.research.google.com/github/Bijan-K/Phi3-notebook/blob/main/phi_3_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Install Core Dependances
!pip -q install torch transformers
!pip -q install accelerate sentencepiece bitsandbytes einops tokenizers optimum

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 44.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.0/417.0 kB 36.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 65.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 49.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.6 MB/s eta 0:00:00


In [ ]:
# @title Install the Model and Translator
import re # for toolcall later on
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from optimum.bettertransformer import BetterTransformer

torch.random.manual_seed(0)

model_phi3 = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-128k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer_phi3 = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct")

tokenizer_madlad = AutoTokenizer.from_pretrained(
    "google/madlad400-3b-mt",
    use_fast=True
)

model_hf = AutoModelForSeq2SeqLM.from_pretrained(
    "google/madlad400-3b-mt",
    torch_dtype=torch.bfloat16
)

model_madlad = BetterTransformer.transform(model_hf, keep_original=True)


In [ ]:
# @title The Pipeline
messages = [
    {"role": "user", "content": """you are a language model called stocki, and are tasked with chatting with the user and being helpful.
Sometimes you have to read the documents they give you and explain it to them in bullet points and also answer questions regarding them.
At other times you have to call tools with arguments extracted from users intent on finnical transitions on stock.
Format your tool call like this: !toolcall: [<users intent>, <type of company>]"""},
]

pipe = pipeline(
    "text-generation",
    model=model_phi3,
    tokenizer=tokenizer_phi3,
)

generation_args = {
    "max_new_tokens": 500,
    "return_full_text": False,
    #"temperature": 0.0,
    #"do_sample": False,
}

def translate_to_english(text):
    text = "<2en> " + text

    inputs = tokenizer_madlad(
        text,
        return_tensors="pt"
    )

    outputs = model_madlad.generate(**inputs, max_new_tokens=1000)
    text_translated = tokenizer_madlad.batch_decode(outputs, skip_special_tokens=True)

    return text_translated[0]

def translate_to_farsi(text):
    text = "<2fa> " + text

    inputs = tokenizer_madlad(
        text,
        return_tensors="pt"
    )

    outputs = model_madlad.generate(**inputs, max_new_tokens=1000)
    text_translated = tokenizer_madlad.batch_decode(outputs, skip_special_tokens=True)

    return text_translated[0]


def reset_session():
  messages = [
      {"role": "user", "content": """you are a language model called stocki, and are tasked with chatting with the user and being helpful.
  Sometimes you have to read the documents they give you and explain it to them in bullet points and also answer questions regarding them.
  At other times you have to call tools with arguments extracted from users intent on finnical transitions on stock.
  Keep your answer short if they are not asking you to explain.
  Format your tool call like this: !toolcall: [<users intent>, <type of company>]"""},
  ]

def call_model_english(input_text):
  messages.append({"role":"user", "content": f"{input_text}"})
  output = pipe(messages, **generation_args)[0]['generated_text']
  messages.append({"role":"assisstant", "content": f"{output}"})
  print(output)
  return output


In [ ]:
# @title Text File
text_file = "Markets exhibit surprising optimism in response to the latest US inflation data. Major US and European indices surge to record highs, reflecting robust investor confidence. Likelihood of ECB rate cuts grows, with potential implementation as early as June. Invest like the big funds from the comfort of your home with our AI-powered ProPicks stock selection tool. Learn more here>> There's no remnant of April's widespread corrective movement on the major US and European indices in sight anymore.  Optimism from the demand side stemmed from Wednesday's US inflation data, aligning with market expectations yet breaking a streak of five consecutive readings above forecasts. It's noteworthy how the market decisively moved without any surprises, reflecting bullish determination to sustain the upward trajectory.  Concurrently, the prospect of the ECB outpacing its overseas counterpart in initiating a series of interest rate cuts is gaining traction.  A series of dovish remarks from European Central Bank officials, particularly Francois Villeroy de Galhau and Mario Centeno, hints at groundwork for a potential 25 basis points cut as early as June.  Let's take a look at three of the most important global indexes to assess their key trading levels at the moment.  S&P 500 Eyeing New All-Time Highs with a Target of at Least 5400 Points This week, the S&P 500 breached new historical highs, albeit marginally. Nevertheless, bullish sentiment suggests an appetite for considerably higher levels, implying a potential continuation of the upward trend with a target near the next significant level of 5400 points." # @param {type:"string"}

In [ ]:
call_model_english(f"please explain the contents of this text: {text_file}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:492: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


 - Markets show optimism due to recent US inflation data.

  - Major US and European stock indices hit record highs.

  - Investor confidence is strong.

- ECB considers rate cuts, possibly in June.

  - Inflation data in the US aligns with market expectations.

  - This is the first time in five readings above forecasts.

- Market movement is decisive and bullish.

  - No surprises in the market's response.

  - Market aims to maintain the upward trend.

- ECB officials hint at interest rate cuts.

  - Francois Villeroy de Galhau and Mario Centeno suggest a cut.

  - The ECB might act before its overseas counterparts.

- S&P 500 index surpasses historical highs.

  - The index marginally surpassed new highs this week.

  - Bullish sentiment indicates a potential rise to at least 5400 points.


If the user asks about using the ProPicks stock selection tool, the tool call would be:

!toolcall: [<explore ProPicks tool>, <stock selection>]


" - Markets show optimism due to recent US inflation data.\n\n  - Major US and European stock indices hit record highs.\n\n  - Investor confidence is strong.\n\n- ECB considers rate cuts, possibly in June.\n\n  - Inflation data in the US aligns with market expectations.\n\n  - This is the first time in five readings above forecasts.\n\n- Market movement is decisive and bullish.\n\n  - No surprises in the market's response.\n\n  - Market aims to maintain the upward trend.\n\n- ECB officials hint at interest rate cuts.\n\n  - Francois Villeroy de Galhau and Mario Centeno suggest a cut.\n\n  - The ECB might act before its overseas counterparts.\n\n- S&P 500 index surpasses historical highs.\n\n  - The index marginally surpassed new highs this week.\n\n  - Bullish sentiment indicates a potential rise to at least 5400 points.\n\n\nIf the user asks about using the ProPicks stock selection tool, the tool call would be:\n\n!toolcall: [<explore ProPicks tool>, <stock selection>]"

In [ ]:
# REGEX
import re

output= """- Understand your interest in the electronics sector.

- Explain the importance of researching and understanding the electronics division before investing.

- Discuss the potential benefits and risks associated with investing in the electronics sector.


If you're looking to get specific information or perform a transaction related to buying shares in the electronics division, I can assist with that. For instance, if you need real-time stock data or a platform to execute the purchase, I would format a tool call like this:

!toolcall: [buy shares, electronics division]"""

regex = r"(!toolcall:.*\n?)"
if re.search(regex, output):
  match = re.search(regex, output)
  print(f"The command: {match.group(0)}")


The command: !toolcall: [buy shares, electronics division]


In [ ]:
reset_session()